In [1]:
import tensorflow as tf
import numpy as np
from matplotlib.pyplot import imshow
from tensorflow.contrib.layers.python.layers import batch_norm
from tensorflow.python.framework import ops
import os
TRAIN = True
INFER = False
BATCH_SIZE = 128
global_step = tf.contrib.framework.get_or_create_global_step()
import urllib, StringIO
from PIL import Image

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [2]:
INFER_URLS = [
    ('deer', 'http://gfp.sd.gov/hunting/big-game/images/deer1-01.jpg'),
    ('frog', 'http://www.gaylordfunkyfish.com/images/Red-Eye-Tree-Frog-300x244.png'),
    ('airplane', 'https://media.licdn.com/mpr/mpr/jc/AAEAAQAAAAAAAAMiAAAAJGVlYTU5Y2YyLWQwMzYtNDlmZS04MDdlLWI0ZjJjZWRhYjk4ZQ.jpg'),
    ('ship', 'http://www.vships.com/media/92241/passenger-vessel.jpg'),
    ('dog', 'https://images-na.ssl-images-amazon.com/images/G/01/img15/pet-products/small-tiles/23695_pets_vertical_store_dogs_small_tile_8._CB312176604_.jpg'),
    ('bird', 'http://www.audubon.org/sites/default/files/styles/engagement_card/public/sfw_apa_2013_28342_232388_briankushner_blue_jay_kk_high.jpg')
]

In [3]:
def read_and_decode_single_example(filename, epochs=None):
    # first construct a queue containing a list of filenames.
    # this lets a user split up there dataset in multiple files to keep
    # size down
    filename_queue = tf.train.string_input_producer([filename],
                                                    num_epochs=epochs)
    # Unlike the TFRecordWriter, the TFRecordReader is symbolic
    reader = tf.TFRecordReader()
    # One can read a single serialized example from a filename
    # serialized_example is a Tensor of type string.
    _, serialized_example = reader.read(filename_queue)
    # The serialized example is converted back to actual values.
    # One needs to describe the format of the objects to be returned
    features = tf.parse_single_example(
        serialized_example,
        features={
            # We know the length of both fields. If not the
            # tf.VarLenFeature could be used
            'label': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], 'string')
        })
    # now return the converted data
    label = features['label']
    image_raw = features['image']
    image = tf.decode_raw(image_raw, tf.uint8)
    image = tf.reshape(image, [32,32,3])
    image = tf.cast(image, tf.float32) / 255
    return label, image

In [9]:
def preprocess(image, distort=False):
    distorted_image = tf.random_crop(image, [24, 24, 3])
    if distort:
        distorted_image = tf.image.random_flip_left_right(distorted_image)
        distorted_image = tf.image.random_brightness(distorted_image, max_delta=0.2)
        distorted_image = tf.image.random_contrast(distorted_image, lower=0.7, upper=1.3)
    distorted_image = tf.image.per_image_standardization(distorted_image) # renamed to per_image_standardization in latest release
    distorted_image = tf.clip_by_value(distorted_image, 0, 1)
    return distorted_image

In [10]:
%matplotlib inline
import urllib2
import matplotlib.pyplot as plt
import PIL
def load_image(url):
    file = StringIO.StringIO(urllib.urlopen(url).read())
    img = Image.open(file)
    img = img.resize((24,24), PIL.Image.BICUBIC)
    img = img.convert('RGB')
    # plt.imshow(img)
    img = (np.array(img.getdata()) / 255.0).reshape((24,24,3))
    # drop last dimension (alpha):
    return img
img = load_image('https://images-na.ssl-images-amazon.com/images/G/01/img15/pet-products/small-tiles/23695_pets_vertical_store_dogs_small_tile_8._CB312176604_.jpg')
# 1

In [11]:
%matplotlib inline

# create image batches:
filename = "cifar-train.tfrecords" if TRAIN else "cifar-test.tfrecords"
label, image = read_and_decode_single_example(filename, epochs=(None if TRAIN else 1))
if TRAIN:
    images_batch, labels_batch = tf.train.shuffle_batch([preprocess(image, True), label], batch_size=BATCH_SIZE, capacity=1000, min_after_dequeue=500)
else:
    images_batch, labels_batch = tf.train.batch([preprocess(image), label], batch_size=BATCH_SIZE, allow_smaller_final_batch=True)


In [12]:
def weight_var(shape, stddev=0.1, weight_decay=0, init=):
    initial = tf.truncated_normal(shape, stddev=stddev)
    v = tf.Variable(initial)
    if weight_decay > 0:
        l2 = tf.nn.l2_loss(v) * weight_decay
        tf.add_to_collection('losses', l2)
    return v


In [16]:
# 16 is a good model
save_path = None # 'models/1'

session = tf.Session()
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
session.run(init_op)

if save_path:
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(save_path)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(session, ckpt.model_checkpoint_path)
        print 'Restored from checkpoint', ckpt.model_checkpoint_path
    else:
        print 'Did not restore from checkpoint'
else:
    print 'Will not save'

Will not save


In [ ]:
tf.train.start_queue_runners(sess=session)

Step: 1; Loss: 2.98754501343; learn rate: 0.0010000000475
Step: 51; Loss: 2.25659036636; learn rate: 0.000999181647785
Step: 101; Loss: 1.9675693512; learn rate: 0.000998363946564


NameError: name 'saver' is not defined